# Mon 1er note book

In [28]:
import pandas as pd
try:
    import yfinance as yf
    print("yfinance a été importé avec succès.")
except Exception as e:
    print(f"Erreur lors de l'importation de yfinance : {e}")

yfinance a été importé avec succès.


In [29]:
df_etf = pd.read_csv("../src/analyse_portefeuille/Liste_ETF.csv", sep=";", encoding="utf-8-sig")
print(df_etf)

    Portefeuille _id          ISIN     Emmeteur  \
0                  1  IE00BF0M2Z96          L&G   
1                  1  IE00BKLF1R75   WisdomTree   
2                  1  IE00BF4TWC33   WisdomTree   
3                  1  IE0002PG6CA6        VnEck   
4                  1  IE000U58J0M1      iShares   
5                  1  IE000FF2EBQ8  BNP Paribas   
6                  1  IE00B5MTYL84      Invesco   
7                  1  IE00B5MJYX09      Invesco   
8                  1  IE00BJXRZJ40         Rize   
9                  1  IE00BYPLS672          L&G   
10                 1  IE000U9ODG19      iShares   
11                 1  IE000YYE6WK5        VnEck   
12                 1  IE00B42NKQ00      iShares   
13                 1  IE00B5MTWH09      Invesco   
14                 2  LU1829221024       Amundi   
15                 3           NaN          NaN   

                                    Nom Devise Type DIST ou ACC  \
0                   Battery Value-Chain    USD  ETF         ACC  

##Test extraction de donnée avec Trakers_justetf puis Trakers_yj

In [30]:
# Liste des tickers que tu veux analyser
tickers = ["BATT.L", "CHRG.L", "XMWJ", "REMX", "INRE"]

# On va juste récupérer les données historiques de ces ETF pour les afficher
for ticker in tickers:
    try:
        # Téléchargement des données
        data = yf.download(ticker, start="2020-06-01", end="2021-01-01")
        
        # Vérification si des données sont disponibles
        if data.empty:
            print(f"Aucune donnée disponible pour {ticker}")
        else:
            # Affichage des premières lignes des données
            print(f"Données de {ticker} :")
            print(data.head())
            
    except Exception as e:
        print(f"Erreur avec {ticker}: {e}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['XMWJ']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Données de BATT.L :
Price        Close     High    Low   Open Volume
Ticker      BATT.L   BATT.L BATT.L BATT.L BATT.L
Date                                            
2020-06-01  8.7370  8.75400  8.613  8.674   5202
2020-06-02  8.9250  8.99700  8.800  8.800  48759
2020-06-03  9.3430  9.31700  9.077  9.153  70919
2020-06-04  9.2850  9.35300  9.181  9.181   5778
2020-06-05  9.8725  9.88195  9.659  9.705   6761
Données de CHRG.L :
Price             Close         High          Low         Open Volume
Ticker           CHRG.L       CHRG.L       CHRG.L       CHRG.L CHRG.L
Date                                                                 
2020-06-01  1895.000000  1895.000000  1895.000000  1895.000000      0
2020-06-02  1916.800049  1919.199951  1902.199951  1919.199951    841
2020-06-03  1913.699951  1900.199951  1895.400024  1895.400024    341
2020-06-04  1915.699951  1924.800049  1909.199951  1920.800049    482
2020-06-05  1961.000000  1943.000000  1943.000000  1943.000000    146
Aucune d

##Extraction des perf des etf sur 1 an

In [39]:
def get_etf_data(ticker, start_date="2020-01-06", end_date="2025-01-01"):
    etf = yf.Ticker(ticker)
    historical_data = etf.history(start=start_date, end=end_date)
    return historical_data

def enrichir_etf_data(row):
    ticker = row['Ticket_yf']
    try:
        historical_data = get_etf_data(ticker, start_date="2024-01-01", end_date="2025-01-01")
        rendement_1an = (historical_data['Close'].iloc[-1] - historical_data['Close'].iloc[0]) / historical_data['Close'].iloc[0] * 100 
        row['Rendement_1an'] = rendement_1an
    except Exception as e:
        row['Rendement_1an'] = None
        print(f"Erreur avec {ticker}: {e}")
    return row

df_etf = df_etf.apply(enrichir_etf_data, axis=1)

# Vérifier les données enrichies
print(df_etf[['Emmeteur','Nom', 'Ticket_yf', 'Rendement_1an']])

Erreur avec nan: 'float' object has no attribute 'upper'
       Emmeteur                                  Nom Ticket_yf  Rendement_1an
0           L&G                  Battery Value-Chain    BATT.L       0.223058
1    WisdomTree                    Battery Solutions    CHRG.L     -11.033965
2    WisdomTree           Industrial Metals Enhanced    META.L       4.424359
3         VnEck      Rare Earth and Strategic Metals    REMX.L     -35.400875
4       iShares                  Global Clean Energy   INRA.AS     -25.496092
5   BNP Paribas  Easy ECPI Global ESG Infrastructure    ENG.PA      10.951953
6       Invesco           European Food & Bev Sector   SC03.DE      -8.556024
7       Invesco          European Industrials Sector   SC0S.DE      15.087767
8          Rize       Cybersecurity and Data Privacy    CYBP.L      13.950009
9           L&G                       Cyber Security    USPY.L      21.350208
10      iShares           Global Aerospace & Defence   DFND.AS      16.290589
11     